# NRL ADCP .mat file to CF-1.6 timeSeriesProfile using pocean

Here we read a matlab file with minimal metadata, and write a CF-DSG 1.6 timeSeriesProfile netcdf file.  We want the file to work seamlessly with ERDDAP, so we add some ERDDAP specific attributes like `cdm_timeseries_variables`, `cdm_profile_variables`, and `subsetVariables`.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from scipy.io import loadmat
import datetime as dt
import numpy as np

#conda install -c conda-forge pocean-core
from pocean.dsg.timeseriesProfile.om import OrthogonalMultidimensionalTimeseriesProfile

In [2]:
# wget http://www.satlab.hawaii.edu/onr/adria/data/moorings/nrl/Final/ADCP_matlab/VR4f.mat
d = loadmat('/data/ADRIA/MOORINGS/NRL/VR4f.mat')

In [3]:
times = [dt.datetime(2002,1,1,0,0,0) + dt.timedelta(a) for a in d['timen'].flatten()]
depths = d['mdepth'].flatten()

# Repeat each time for the number of depths
t = np.repeat(times, len(depths))

# Create a profile index, and repeat for number of depths
profile = np.repeat(np.array(range(len(times)), dtype=np.int32) + 1, len(depths))

# Tile the depths for each time
z = np.tile(depths, len(times))

df = pd.DataFrame({
    't': t,
    'x': 13.0281,
    'y': 45.187783,
    'z': z,
    'un': d['un'].T.flatten()/10.,   # cm/s to m/s
    'vn': d['vn'].T.flatten()/10.,   # cm/s to m/s
    'wn': d['wn'].T.flatten()/10.,   # cm/s to m/s
    'profile': profile,
    'station': 'VR4F'
})

df.tail()

,profile,station,t,un,vn,wn,x,y,z
296338,5199,VR4F,2003-04-29 03:00:00,0.061417,0.394078,0.017406,13.0281,45.187783,28.548073
296339,5199,VR4F,2003-04-29 03:00:00,-0.044268,0.653439,0.003686,13.0281,45.187783,29.048073
296340,5199,VR4F,2003-04-29 03:00:00,0.054443,0.386804,0.004221,13.0281,45.187783,29.548073
296341,5199,VR4F,2003-04-29 03:00:00,0.098836,0.529064,-0.011401,13.0281,45.187783,30.048073
296342,5199,VR4F,2003-04-29 03:00:00,0.008518,0.550976,0.011823,13.0281,45.187783,30.548073


In [4]:
atts={
            'global': {
                'title': 'ADRIA02 Mooring VR4',
                'summary': 'Data from bottom-mounted ADCP',
                'institution': 'NRL',
                'cdm_timeseries_variables': 'station',
                'cdm_profile_variables': 'profile',
                'subsetVariables': 'depth'
            },
            'longitude': {
                'units': 'degrees_east',
                'standard_name':'longitude'
            },
            'latitude': {
                'units': 'degrees_north',
                'standard_name':'latitude'
            },
            'z': {
                'units': 'm',
                'standard_name': 'depth',
                'positive':'down'
            },
            'un': {
                'units': 'm/s',
                'standard_name':'eastward_sea_water_velocity'
            },
            'vn': {
                'units': 'm/s',
                'standard_name':'northward_sea_water_velocity'
            },
            'profile': {
                'cf_role': 'profile_id'
            }
        }

In [5]:
atts['global']

{'cdm_profile_variables': 'profile',
 'cdm_timeseries_variables': 'station',
 'institution': 'NRL',
 'subsetVariables': 'depth',
 'summary': 'Data from bottom-mounted ADCP',
 'title': 'ADRIA02 Mooring VR4'}

In [6]:
OrthogonalMultidimensionalTimeseriesProfile.from_dataframe(df, output='/data/ADRIA/MOORINGS/NRL/vr4f.nc', 
                                                          attributes=atts)

/home/richard.signell/miniconda3/envs/IOOS3/lib/python3.6/site-packages/pocean/dsg/timeseriesProfile/om.py:82: UserWarning: endian-ness of dtype and endian kwarg do not match, using endian kwarg
  latitude = nc.createVariable('latitude', get_dtype(df.y), ('station',))
/home/richard.signell/miniconda3/envs/IOOS3/lib/python3.6/site-packages/pocean/dsg/timeseriesProfile/om.py:83: UserWarning: endian-ness of dtype and endian kwarg do not match, using endian kwarg
  longitude = nc.createVariable('longitude', get_dtype(df.x), ('station',))
/home/richard.signell/miniconda3/envs/IOOS3/lib/python3.6/site-packages/pocean/dsg/timeseriesProfile/om.py:84: UserWarning: endian-ness of dtype and endian kwarg do not match, using endian kwarg
  z = nc.createVariable('z', get_dtype(df.z), ('z',))
/home/richard.signell/miniconda3/envs/IOOS3/lib/python3.6/site-packages/pocean/dsg/timeseriesProfile/om.py:108: UserWarning: endian-ness of dtype and endian kwarg do not match, using endian kwarg
  v = nc.create

<class 'pocean.dsg.timeseriesProfile.om.OrthogonalMultidimensionalTimeseriesProfile'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6
    date_created: 2017-06-21T12:19:00Z
    featureType: timeseriesProfile
    cdm_data_type: TimeseriesProfile
    title: ADRIA02 Mooring VR4
    summary: Data from bottom-mounted ADCP
    institution: NRL
    cdm_timeseries_variables: station
    cdm_profile_variables: profile
    subsetVariables: depth
    dimensions(sizes): station(1), time(5199), z(57)
    variables(dimensions): int32 crs(), <class 'str'> station(station), float64 time(time), float64 latitude(station), float64 longitude(station), float64 z(z), int32 profile(time,z,station), float64 un(time,z,station), float64 vn(time,z,station), float64 wn(time,z,station)
    groups: 